In [1]:
!pip install transformers sentencepiece
!pip install nltk
!pip install rank-bm25
!pip install faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 95.5 MB/s eta 0:00:00


In [1]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 44.1 MB/s eta 0:00:00


In [5]:
import uuid
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [6]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # placeholder
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

SUM_MODEL = "google/flan-t5-small"
sum_tokenizer = AutoTokenizer.from_pretrained(SUM_MODEL)
sum_model = AutoModelForSeq2SeqLM.from_pretrained(SUM_MODEL).to("cuda")

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [2]:
from sentence_transformers import SentenceTransformer

EMBED_MODEL_NAME = "intfloat/e5-small"
embed_model = SentenceTransformer(EMBED_MODEL_NAME, device="cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/362 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [7]:
CHUNK_SIZE = 400
OVERLAP = 40

In [8]:
def count_tokens(text):
    return len(tokenizer.encode(text))

In [9]:
def parser(text, source):
    raw_section = text.split('\n\n')
    sections = []
    for idx, sec in enumerate(raw_section):
        sec = sec.strip()
        if len(sec) == 0:
            continue

        sections.append({
            "section_id": str(uuid.uuid4()),
            "title": f"section_{idx}",
            "level": 0,
            "content": sec
        })
    return {
        "doc_id": str(uuid.uuid4()),
        "source": source,
        "sections": sections
    }


In [70]:
def chunk_sections(section, doc_id):
    content = section["content"]

    tokens = tokenizer.encode(content, add_special_tokens=False)
    chunks = []
    start = 0
    position = 0

    while start < len(tokens):
        end = start + CHUNK_SIZE
        chunk_tokens = tokens[start:end]

        chunk_text = clean_text(tokenizer.decode(chunk_tokens))

        chunks.append({
            "chunk_id": str(uuid.uuid4()),
            "doc_id": doc_id,
            "section_id": section["section_id"],
            "position_index": position,
            "content": chunk_text,
            "token_count": len(chunk_tokens),
            "overlap_from_previous": OVERLAP if position > 0 else 0
        })

        start += CHUNK_SIZE - OVERLAP
        position += 1
    return chunks


In [77]:
def generate_summary(text, max_tokens=60):
    if count_tokens(text) < 120:
        return text
    prompt = f"""
    Extract the core technical concepts and main topics from the following text.
    Return a short dense description (max 40 tokens).
    Do not repeat the text. Do not include metadata.

    Text:
    {text}
    """

    inputs = sum_tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to("cuda")

    outputs = sum_model.generate(
      **inputs,
      max_new_tokens=40,
      do_sample=True,
      temperature=0.3,
      top_p=0.9
  )

    summary = sum_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary.strip()


In [12]:
def ingest_text_pipeline(text, source="unknown"):
    doc = parser(text, source)

    all_chunks = []

    for chunk in doc["sections"]:
        section_chunking = chunk_sections(chunk, doc["doc_id"])

        for section_chunk in section_chunking:
            summary = generate_summary(section_chunk['content'])
            section_chunk['summary'] = summary

        all_chunks.extend(section_chunking)

    return {
        'doc': doc,
        'chunks': all_chunks
    }



In [71]:
with open("/content/sample.txt", "r") as f:
    text_data = f.read()

In [14]:
print(text_data[:100])

Machine learning (ML) is a field of study in artificial intelligence concerned with the development 


In [72]:
result = ingest_text_pipeline(text_data, 'Wikipedia Machine Learning')

In [15]:
print(result["chunks"][0])

{'chunk_id': 'fc025a1b-29d7-441f-8d88-13f66f80110e', 'doc_id': 'bb0c6bbd-7646-4e6c-97f0-157112804540', 'section_id': '1c9c2a74-7d33-4518-919a-40a97aae9d87', 'position_index': 0, 'content': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.', 'token_count': 85, 'overlap_from_previous': 0, 'summary': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine 

In [61]:
def embed_passages(chunks):
    texts = [f"passage: {chunk['content']}" for chunk in chunks]

    embeddings = embed_model.encode(
        texts,
        batch_size=32,
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    return embeddings


In [17]:
def embed_query(query):
    return embed_model.encode(
        f"query: {query}",
        convert_to_numpy=True,
        normalize_embeddings=True
    )

In [18]:
import faiss
import numpy as np

def build_faiss_index(embeddings):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)  # cosine similarity (since normalized)
    index.add(embeddings)
    return index


In [66]:
def retrieve(query, index, chunks, top_k=3):
    query_vec = embed_query(query)
    scores, indices = index.search(query_vec.reshape(1, -1), top_k)

    results = []
    for idx in indices[0]:
        results.append(chunks[idx])

    return results


In [69]:
import re

def clean_text(text):
    text = re.sub(r"\[\d+\]", "", text)  # remove citation markers
    text = re.sub(r"\s+", " ", text)     # normalize whitespace
    return text.strip()

In [73]:
chunks = result["chunks"]
embeddings = embed_passages(chunks)
fias_index = build_faiss_index(embeddings)

In [74]:
fias_index.ntotal

24

In [46]:
a = ['How does the "operational definition" of machine learning provided by Tom Mitchell bridge the gap between Alan Turing’s mid-century philosophical inquiries and the early 1960s experimental "learning machines" like the Raytheon Cybertron?',
     'The text mentions that most machine learning can be described as "empirical risk minimisation." Based on the sections regarding Optimisation and Data Mining, how does the role of a "loss function" differentiate the performance evaluation of a standard ML model from the core objective of Knowledge Discovery in Databases (KDD)?',
     """According to the provided text, why might the Chinchilla 70B model's success in outperforming PNG and FLAC compression formats be considered a "false positive" for general intelligence or compression efficiency?""",
     """Compare the "theoretical neural structure" proposed by Donald Hebb in 1949 with the "connectionism" movement of the 1980s. Which specific mathematical breakthrough allowed the latter to overcome the "logic-based approach" that had previously caused a rift between AI and statistics?""",
     """Identify a specific unsupervised learning technique mentioned in the text that serves as a dual-purpose tool for both exploratory data analysis and enhancing storage efficiency in signal processing. Explain the mechanism (the "how") it uses to achieve the latter."""]

In [75]:
for i in a:
    results = retrieve(i, fias_index, chunks, 3)
    print("\nQUERY:", i[:80])
    for r in results:
        print("----")
        print(r["content"][:200])
        print("Summary:", r.get("summary"))


QUERY: How does the "operational definition" of machine learning provided by Tom Mitche
----
Tom M. Mitchell provided a widely quoted, more formal definition of the algorithms studied in the machine learning field: "A computer program is said to learn from experience E with respect to some cl
Summary: Can machines do what we (as thinking entities) can do what we (as thinking entities) can do what we (as thinking entities) can do what we (as thinking entities
----
By the early 1960s, an experimental "learning machine" with punched tape memory, called Cybertron, had been developed by Raytheon Company to analyse sonar signals, electrocardiograms, and speech patte
Summary: Nils Nilsson's "learning machine" with punched tape memory, called Cybertron, had been developed by Raytheon Company to analyse sonar signals, electrocardiograms,
----
Relationships to other fields Artificial intelligence
Summary: Relationships to other fields Artificial intelligence

QUERY: The text mentions that most 